# Capitulo 2. Proyecto de Machine Learning

👀: verifica que si hayas instalado las librerías que vas a necesitar

En este caso se realizará un modelo para predecir la media de precios en las viviendas de los diferentes municipos de Londres, Reino Unido 🇬🇧.

## <span style="color:green">1. Descargar los datos</span>

Las bases de datos para este proyecto se puede encontrar en este enlace: https://www.kaggle.com/justinas/housing-in-london

También se pueden consultar todas las bases de datos de este curso en GitHub:https://github.com/a2Proyectos/MachineLearning_Data

- housing_in_london_yearly_variables.csv, con los datos que necesitamos para hacer la regresión.
- London_Borough_Excluding_MHW.shp, con los datos que necesitamos para graficar Londres.
- Capitulo_2/housing_in_london_monthly_variables.csv, con los datos de la media salarial

### Queremos adivinar los precios de la vivienda 

División por distritos (Borough Flag) nos indica si es de Londres o de las afueras

Antes: Una persona, analisis de datos + formulas y estadística --> Decidir si invertir o no (25% sesgo)
Ahora: ML --> Decido si invertir o no (Si mi sesgo es menor de 25%, mi ML es útil)

#### Que nos tenemos que preguntar a la hora de enfrentarnos al análisis de un DataSet:

    ¿Aprendizaje supervisado o no?
        Nos vamos a fijar en si tenemos etiquetas o no.
            Como tenemos etiquetas, aplicaremos aprendizaje supervisado.

    ¿Regresión o clasificación?
        Que queremos hacer, predecir o clasificar...
            En nuestro caso queremos predecir el precio de una vivienda por lo que usaremos regresión.

            Regresión, ¿univariable o multivariable?
                Solo quiero que me devuelva el dato del precio de la vivienda por lo que univariable.
    
    ¿Batch u Online?
        Trabajar los datos en local o recibir los datos actualizados siempre que los pido
            Batch(Con variables locales guardando los datos para manipularlos en éstas)
    
    ¿Qué necesita mi equipo?
        Tengo que definir(y discutir) que me está pidiendo mi equipo. (quieren que devuelva un valor o una clasificación).
            Dependiendo de lo que nos pidan tendríamos que reconfigurar la salida de nuestro algoritmo.

In [ ]:
## Importamos nuestras librerias principales panda, numpy, matplotlib, os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
## Definimos una función para extraer datos.
#DOWNLOAD_ROOT es la base del GitHub donde vamos a estar descargando las bases de datos.
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/a2Proyectos/MachineLearning_Data/main/"
#Complementos con la dirección especifica de la base de datos que queremos.
LONDON_SALARY = "Capitulo_2/housing_in_london_yearly_variables.csv"
LONDON_HOUSING = "Capitulo_2/housing_in_london_monthly_variables.csv"
LONDON_MAP =  "https://raw.githubusercontent.com/a2Proyectos/MachineLearning_Data/main/Capitulo_2/map/London_Borough_Excluding_MHW.shp"


def extraer_datos(root,database):
    csv_path = root + database
    return pd.read_csv(csv_path)

***

## <span style="color:green">2. Vistazo a la Base de Datos</span>

In [ ]:
## Juntamos nuestra base de datos de la media salarial, con la de datos de Londres.
df1 = extraer_datos(DOWNLOAD_ROOT,LONDON_SALARY)
df2 = extraer_datos(DOWNLOAD_ROOT,LONDON_HOUSING)


In [ ]:
df1

In [ ]:
df2

In [ ]:
#Filtrar los datos 
df1 = df1.filter(items=["median_salary","area","date"])
df1.head()


In [ ]:
#Fusionar los dos dataframe
data = pd.merge(df1,df2)
data.head()



In [ ]:
# Obtener información de los datos.
data.info()

👀:Comienza a ver el tipo de variable, de eso depende el análisis que le demos o si necesitamos cambiarlas a otro tipo de variable

***

In [ ]:
# Una forma muy común para saber que tipo de datos contiene alguna variable de tipo objeto, es contar sus valores, ejemplo:
data["area"].value_counts()

In [ ]:
#Una forma muy utilizada para obtener información de nuestros datos numéricos es el método describe(), ejemplo:
pd.options.display.float_format = "{:,.2f}".format
data.describe()

In [ ]:
# Realiza un Histograma para visualizar los datos 
#matplotlib inline para aquellos que estan con jupyter notebook
#%matplotlib inline

data.hist(bins=50,figsize=(15,10))
plt.show()


***

## <span style="color:green">3. Crear un set de entrenamiento y de prueba </span>

Recuerden que nuestro set de prueba lo pondremos de lado por ahora, sin verlo. No sean tramposos ❌

In [1]:
#Importamos la función para dividir los datos train_test_split
from sklearn.model_selection import train_test_split

In [3]:
# Seleccionamos por ejemplo, el 30% de los datos para el set de prueba
#Random_state es la semilla que se usa para generar números aleatorios.
set_ent, set_prueba = train_test_split(data,test_size=0.3,random_state=11)


NameError: name 'data' is not defined

In [ ]:
print(len(set_ent),len(set_prueba))

Ya tenemos nuestro set de prueba y nuestro set de entrenamiento ✅

***

### <span style="color:blue">3.1 Evitar Sesgo </span>

In [4]:
# Para categorizar una variable, ejemplo con 5 niveles. 
data["salary_cat"] = pd.cut(
    data["median_salary"],  # variable continua
    bins=[0.,10000,20000,30000,40000,np.inf],                     # 5 niveles
    labels=["Muy Bajo", "Bajo", "Medio", "Alto", "Muy Alto"]
)
                           
#Hacer un histograma de las categorías

data["salary_cat"].hist()

NameError: name 'pd' is not defined

In [ ]:
#Verificar que no existen datos en el bin #1
data["salary_cat"].value_counts()

In [ ]:
# Redefinir en 4 categorías, quitando en donde no hay datos (el bin#1)
data = data.dropna(subset=["median_salary"])
data = data.reset_index(inplace=True)
data["salary_cat"] =pd.cut(data["meadian_salart"],bins=[10000,20000,30000,40000,np.inf],labels=[1,2,3,4])
data["salary_cat"].hist()

In [ ]:
#Verifica que si se redefinieron las categorías 
data["salary_cat"].value_counts

In [ ]:
## Dividir datos basándonos en nuestras categorías de salarios
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
# Generamos nuestro objeto para que lo divida en 30% y solo haga una división
split = StratifiedShuffleSplit(n_splits=1,test_size=0.3,random_state=11)

In [ ]:
# Creamos nuestras variables basándonos en nuestras categorías
for ent_index, prueba_index in split.split(data, data["salary_cat"]):#Definir de nuevo, entrenamiento y prueba peroteniedno en cuenta las nuevas etiquetas para eliminar sesgo
    cat_set_ent = data.loc[ent_index]
    cat_set_prueba = data.loc[prueba_index]


In [ ]:
# Comprobación. Ya en porcentaje
cat_set_prueba["salary_cat"].value_counts()/len(cat_set_prueba)

***

In [ ]:
# Creamos dataframe para trabajar con el set de entrenamiento
df = cat_set_ent.copy()
df.head()

***

## <span style="color:green">4. Visualizar los Datos Gráficamente </span>


Para esto vamos a necesitar, en conjunto con nuestro dataset LONDON_MAP, una nueva libreria que se llama geopandas, el cual exteiende la libreria pandas, para trabajar con datos geoespaciales, se puede encontrar más información en: https://geopandas.org/getting_started/introduction.html

In [ ]:
#Importar geopandas
import geopandas as gpd

#Leer el mapa
londres_map =gpd.read_file(LONDON_MAP)
londres_map.head()

In [ ]:
# Graficamos el mapa
londres_map.plot()

***

In [ ]:
# Ajustamos los nombres de las columnas para despúes hacer un merge.
#utilizar lower para cambiar de mayúsculas a minúsculas 
londres_map.colums = londres_map.columns.str.lower()
londres_map = londres_map.rename({"name":"area","gss_code":"code"},axis=1)
londres_map["area"] = londres_map["area"].str.lower()

#Seleccionar columnas necesarias
londres_map = londres_map.filter(items=["area","code","hectares","geometry"])
londres_map.head()

***

In [ ]:
# Seleccionar datos de nuestro set de entrenamiento
df_m = df.groupby("area").agg({"avarage_price":["mean"],"houses_sold":"sum"})

# Le asignamos nombre a las columnas del nuevo dataframe y reseteamos el indice
df_m.columns = ["avarage_price", "houses_sold"]
df_m.reset_index(inplace=True)
df_m.head()

In [ ]:
# Combinar dtaframes
londres_map = pd.merge(londres_map,df_m,on="area")
londres_map.head()

In [ ]:
londres_map.plot?

***

In [ ]:
#Gráfica del promedio de los precios en las casas 
#Cuando se grafica en geopandas hay muchos argumentos, no se desesperen si no los recuerdan, es normal. 
plt = londres_map.plot(column = "average_price", #Columna en la que se basa la gráfica
                        cmap = "Reds", #Color gradiente de la gráfica
                        edgecolor="maroon", #Color de contorno
                        legend = True, legend_kwds ={"label":"Precio","orientation":"horizontal"}) #leyenda
plt.set_title("Media de los precios en las casas")
plt.axis("off")

💸: Recuerden que los precios son mayores en el centro de la cuidad

In [ ]:
#Graficar ahora el total de las casa vendidas (utilizar el código anterior para no repetir)
plt = londres_map.plot(column = "houses_sold", #Columna en la que se basa la gráfica
                        cmap = "Blues", #Color gradiente de la gráfica
                        edgecolor="maroon", #Color de contorno
                        legend = True, legend_kwds ={"label":"Precio","orientation":"horizontal"}) #leyenda
plt.set_title("Total de casas vendidas")
plt.axis("off")

In [ ]:
londres_map["median_salary"]=df["median_salary"]

plt = londres_map.plot(column = "median_salary", #Columna en la que se basa la gráfica
                        cmap = "Greens", #Color gradiente de la gráfica
                        edgecolor="black", #Color de contorno
                        legend = True, legend_kwds ={"label":"Precio","orientation":"horizontal"}) #leyenda
plt.set_title("Total de casas vendidas")
plt.axis("off")

***

## <span style="color:green">5. Medir la Correlación </span>


In [ ]:
# Crear matriz de correlación


# Comparar correlación


In [ ]:
#Importar seaborn 


# Crear vector


# Graficar


In [ ]:
#Importar pandas 


#Para graficar scatter_matrix...


In [ ]:
#Por si quieren ver una gráfica en específico más a detalle


***

## <span style="color:green">6. Combinación de Variables </span>


In [ ]:
#Armar una columna para hacer las combinaciones que necesitamos

# Crear matriz de correlación


***

***

## <span style="color:Blue">7. Transformación de Datos </span>


In [ ]:
# Crear Dataframe de predictores y variable a predecir ✂️

#Nuevo dataframe sin average_price


***

In [ ]:
#Darte una idea de a qué variables le hace faltan datos

#Calcular la suma de todos los vacíos  


In [ ]:
#Quitar el número de crímenes por su alta cantidad de datos vacíos 


In [ ]:
#Tomamos la media 

#Llenamos los valores con la media

#Verifica que no hay datos vacíos


***

Recuerden que volvimos a ejecutar la línea de código de "df = cat_set_ent.drop('average_price', axis=1)" para hacer este ejercicio

In [ ]:
#BIENVENIDOS A SCIKIT <3 Serán unos expertos al final. 
# Rellenar valores con scikit
#1. Recuerden: importar lo que vayas a utiliza, en este caso SimpleImputer


# 2. Crear objeto, en este caso se llamará imputer



# data numérico


#Ejecuta Imputer


#Aplicar transform para rellenar las medianas 


#Regresarlo a dataframe 


***

## <span style="color:Blue">8. Manejo de texto y valores categóricos </span>

In [ ]:
#definir que variable vamos a cambiar a valor numérico

#Convertir variables de texto en numéricas

#verificar que si se haya transformado


#Por si quieres ver como funciona el encoder


In [ ]:
#ONE HOT ENCODER, es importante recordar y entender esta función porque la usaremos en todo el curso
#Convertir variables categóricas en binarias
#Importar OneHotEncoder

#Crear objeto

#Ajustar

#obligar a que nos muestre la matriz 


***

## <span style="color:Blue">9. Escalación de variables </span>


#### <span style="color:Blue">9.1 Normalización </span>

In [ ]:
#Importar MinMaxScaler

#Creamos el objeto 

#Hacer un dataframe con la transformación


#### <span style="color:Blue">9.2 Estandarización </span>

In [ ]:
## Estandarización (Es el más ultizado)
#Importar StandardScaler (presten especial atención a StandardScaler lo estaremos viendo muy seguido) 

#Crea objeto

#Hacer un dataframe con la transformación


***

## <span style="color:Blue">10. Pipeline </span>


In [ ]:
# Crear función de pipeline


In [ ]:
# Column Transformer (nos permite transformar varias columnas al mismo tiempo y luego juntarlas)

#Hacer un datframe que usaremos para la regresión linea

#Visualiza los datos 


***

***

## <span style="color:purple">11. Seleccionar y entrenar modelos </span>

In [ ]:
## Hacer Regresión Lineal (AL FIN)


In [ ]:
#Análisis preambultario de los errores


***

#### <span style="color:purple">11.1 RMSE </span>

In [ ]:
# Calcular el promedio de la suma de los errores al cuadrado RMSE


In [ ]:
#Sacar el promedio 


In [ ]:
#Calcular el porcentaje de acierto


***

#### <span style="color:purple">11.2 Árbol de Decisión </span>

In [ ]:
# importar DecisionTreeRegressor

#crear objeto 

#correrlo


#calcular el error


***

#### <span style="color:purple">11.3 Validación Cruzada </span>

¡Anota esto porque es importante!

In [ ]:
#Importar cross_val_score, creo objeto, corro mi función


In [ ]:
#Calcular el porcentaje de acierto de validación cruzada 


***

#### <span style="color:purple">11.4 Bosque Aleatorio </span>
¡Este también es importante!

In [ ]:
#Importar RandomForestRegressor, creo objeto, corro mi función


#calcular el error


In [ ]:
#Calcular el el promedio de la suma de los errores al cuadrado


In [ ]:
#Calcular el porcentaje de acierto de bosque aleatorio


***

## <span style="color:purple">12. Afinar el modelo </span>

### <span style="color:purple">12.1 Grid Search</span>

In [ ]:
#Importar GridSearchCV


### <span style="color:purple">12.2 set de prueba</span>

In [ ]:
#¡AHORA SI! A utilizar el set de prueba. Es la última parte 

#Define cuál es el modelo final 


#Aquí definimos cuál será nuestra variable a predecir y las predichas 



#Ahora, sí. El pipeline para limpieza de datos 



#Por último, ver el error del modelo 


In [ ]:
#Ver el porcentaje de acierto
